<a href="https://colab.research.google.com/github/farshidehkordi/Homework2_AI/blob/main/TP2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os

# List files in the current directory
print(os.listdir())


['.config', 'drive', 'sample_data']


In [8]:
!pwd

/content


In [9]:
!ls /content/drive/MyDrive/

'~$فصل-سمینار.docx'
'01_Ou Sont Les Hommes.mp3'
"02_Ceux Qui N' Ont Rien.mp3"
'02. Koudaki.mp3'
 2020-08-31_8-38-46.mp4
 Chapter3.mp3
 Chapter4.mp3
'Colab Notebooks'
'cover (1).zip'
 cover.zip
 CUB_200_2011.tar
 farshideh_kordi_12026250_18524408.pdf
 first_writing.docx
 image.JPG
 IMG_1285.JPG
'information of university (1).gdoc'
'information of university.docx'
'information of university.gdoc'
 kordi.pdf
 Lhokm0.tif
 mona.mp4
 MsForm12.docx
 MsForm12.gdoc
 MWSCAS2023_CFP_v3.gdoc
 Oxford.Word.Skills.Basic-CD-ROM_IELTSMatters.com.zip
 paper.rar
 photo_۲۰۱۷-۰۴-۲۶_۱۱-۲۱-۰۸.jpg
 photo_۲۰۱۷-۰۴-۲۶_۱۱-۲۱-۰۸.jpg.gdoc
 photo_۲۰۱۷-۰۴-۲۶_۱۱-۲۱-۱۱.jpg
 photo_۲۰۱۷-۰۴-۲۶_۱۱-۲۱-۱۱.jpg.gdoc
 Power-Analysis-Attacks-Revealing-the-Secrets-of-Smart-Cards-Advances-in-Information-Security-.pdf
 power_attack.rar
 PowerTraces.rar
 RiaziMohandesiKarimi.pdf
 Tarck06.mp3
 Track05.mp3
 Track06.mp3
 Track07.mp3
 Track08.mp3
 Track09.mp3
 Track10.mp3
 Track11.mp3
 Track12.mp3
 Track13.mp3
 Track14.mp3
 Track15.mp3


In [10]:
import os

# List files in the current directory
print(os.listdir())

# Check if the uploaded file exists
if 'CUB_200_2011.tar' in os.listdir():
    print("File exists!")
else:
    print("File not found.")


['.config', 'drive', 'sample_data']
File not found.


In [12]:
import tarfile

# open file
file = tarfile.open('/content/drive/MyDrive/CUB_200_2011.tar')



# extract files
file.extractall('./CUB_dataset')

# close file
file.close()


In [13]:
import os
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from shutil import copyfile




# Function to create a directory if it doesn't exist
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# Function to separate images into training and test sets
def separate_train_test(dataset_path, train_path, test_path):
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        if classname.startswith('.'):
            continue
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.startswith('.'):
                continue
            file_path = os.path.join(dataset_path, classname, file)
            if i < 15:  # Number of images for the test set
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1
        class_index += 1







In [14]:
dataset_path = './CUB_dataset/CUB_200_2011/images'
train_path = './CUB_dataset/train'
test_path  = './CUB_dataset/test'

# Call the function to separate the dataset into training and test sets
separate_train_test(dataset_path, train_path, test_path)

ResNet18 using random initialization.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with random initialization
model =resnet18(weights=None)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


ResNet18 using pretrained model, but freezing all the convolution parameters

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50 , ResNet50_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with pre-trained weights
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)


# Freeze all the convolution parameters
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layer with a new one (unfrozen)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)  # Only optimize the fully connected layer

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


KeyboardInterrupt: 

Resnet18 using random initialization by default for ImageNet dataset

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load the ImageNet dataset
train_dataset = datasets.ImageNet(root='./data', split='train', transform=transform)
val_dataset = datasets.ImageNet(root='./data', split='val', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the ResNet-18 model with random initialization
model = models.resnet18(pretrained=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')
